In [18]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [2]:
load_dotenv()



True

In [4]:
model = ChatGroq(model="Gemma2-9b-It")

In [6]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="Score out of 10", ge=0, le=10)

In [7]:
structured_model = model.with_structured_output(EvaluationSchema)

In [16]:
essay = """Global warmming is like when its hot outside but like not the normal hot its more hotter than before. So people say its a problem but like also its nature and maybe nature just doing its thing. I saw on TV that like the polar bears are sad, which is not cool but also how do we know they are sad? Maybe they like it warm? Idk.

Anyways, carbon dioxside is like something cars do and when theres more cars, theres more carbon and it goes up and then its bad. But also planes do that too. And cows? Cows fart and that makes climate worst which is funny cause cows are cute lol. If we stop cows then no burgers, and that would be also bad so it's complicated.

Some people say recycle and turn off lights and stuff, but also some people still use plastic so it's kinda like why even bother? The earth is big and people are small so it can’t really matter that much if I throw away a bottle, right?

In conclusion, global warming is like a thing that is happening or maybe not, it depends. Some say it's fake, some say it's real, and the truth is probably somewhere in the middle. The ice is melting but maybe it'll stop melting later. We just have to wait and see what happens I guess.

"""


In [17]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
structured_model.invoke(prompt)


EvaluationSchema(feedback='The essay demonstrates a basic understanding of global warming but lacks depth and accuracy. The language is informal and contains grammatical errors. The tone is dismissive and uncertain, failing to convey the seriousness of the issue.  Recommendations include:  - using more precise language and scientific terminology; - providing more factual information and evidence; - adopting a more serious and informative tone; - exploring the issue in greater detail, discussing potential solutions and consequences.', score=4)

In [20]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [21]:
def evaluate_language(state: UPSCState):
    essay = state['essay']

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {'language_feedback':output.feedback, "individual_scores":[output.score]}


def evaluate_analysis(state: UPSCState):
    essay = state['essay']

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {'analysis_feedback':output.feedback, "individual_scores":[output.score]}


def evaluate_thought(state: UPSCState):
    essay = state['essay']

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {'clarity_feedback':output.feedback, "individual_scores":[output.score]}


def final_evaluation(state: UPSCState):
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

    


In [22]:
graph = StateGraph(UPSCState)

graph.add_node("evaluate_language",evaluate_language)
graph.add_node("evaluate_analysis",evaluate_analysis)
graph.add_node("evaluate_thought",evaluate_thought)
graph.add_node("final_evaluation",final_evaluation)
